In [2]:
import pandas as pd

In [11]:
tda = pd.read_csv('/mnt/resource/m.schepers/Data/TDA_Data/Training_data_run1.csv')
print(tda.shape)

(361, 459)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from scipy import stats
from sklearn import linear_model
import statsmodels.api as sm
import numpy as np
from matplotlib.offsetbox import AnchoredText
import statsmodels.formula.api as smf

In [2]:
bd = pd.read_csv('/mnt/resource/m.schepers/Data/Cog_Data/Cog_Females_train.csv')
tda = pd.read_csv('/mnt/resource/m.schepers/Data/TDA_Data/Training_data_run1.csv')

In [4]:
# Only include subjects from tda which are also in bd
same = tda['Subject'].isin(bd['subject'])
print(same.value_counts())
tda.drop(tda[~same].index, inplace = True)
print(tda.shape)
tda.sort_values(by=['Subject'], inplace=True)
tda.reset_index(drop=True, inplace=True)

True    203
Name: Subject, dtype: int64
(203, 459)


In [8]:
# Only include subjects from bd which are also in tda
same = bd['subject'].isin(tda['Subject'])
print(same.value_counts())
bd.drop(bd[~same].index, inplace = True)
print(bd.shape)
bd.sort_values(by=['subject'], inplace=True)
bd.reset_index(drop=True, inplace=True)

True    203
Name: subject, dtype: int64
(203, 41)


In [9]:
# Concatenate bd and tda into df
print(bd.shape)
print(tda.shape)
df = pd.concat([bd, tda], axis=1)
print(df.shape)

(203, 41)
(203, 459)
(203, 500)


In [10]:
# Create EF (executive functioning column) as mean of flanker and dccs
df['EF_age'] = df[['flanker_age', 'dccs_age']].mean(axis=1)
df['EF_unadjusted'] = df[['flanker_unadjusted', 'dccs_unadjusted']].mean(axis=1)

# Order bd columns in lists
age_adjusted = [i for i in bd.columns if '_age' in i]
age_adjusted.append('EF_age')
age_adjusted.append('age_years')
unadjusted = [i for i in bd.columns if '_unadjusted' in i]
unadjusted.append('EF_unadjusted')
natper = [i for i in bd.columns if '_np' in i]
subject_information = ['subject',  'sex', 'age_months']
WM = ['lswmt_unadjusted', 'lswmt_age']
EF = ['dccs_unadjusted', 'dccs_age',
       'flanker_unadjusted', 'flanker_age',
      'EF_unadjusted', 'EF_age']

In [11]:
# Order tda columns in lists
tda_features = tda.columns[1:]

DMN_curvatures = [i for i in tda_features if 'DMN' in i if 'curv' in i]
FPN_curvatures = [i for i in tda_features if 'FPN' in i if 'curv' in i]
all_curvatures = [i for i in tda_features if 'all' in i if 'curv' in i]
p3_cliques = [i for i in tda_features if 'p3cliques' in i]
p4_cliques = [i for i in tda_features if 'p4cliques' in i]
global_features = [i for i in tda_features if i not in DMN_curvatures if i not in FPN_curvatures if i not in all_curvatures if i not in p3_cliques if i not in p4_cliques]

WM_age = ['lswmt_age']
EF_age = ['dccs_age', 'flanker_age', 'EF_age']
all_kur = [i for i in tda_features if 'kur' in i]
all_skew = [i for i in tda_features if 'skew' in i]
all_std = [i for i in tda_features if 'std' in i]
all_ent = [i for i in tda_features if 'curv_ent' in i]

In [12]:
to_include = age_adjusted + unadjusted

In [13]:
# Only include age_adjusted, and then add subject column
df_no_nan = df[to_include]
df_no_nan['subject'] = list(df['subject'])
df_no_nan['EF_unadjusted'] = list(df['EF_unadjusted'])
print(df_no_nan.shape)

(203, 26)


<ipython-input-13-3ffb2cdb13d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_nan['subject'] = list(df['subject'])
<ipython-input-13-3ffb2cdb13d0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_nan['EF_unadjusted'] = list(df['EF_unadjusted'])


In [14]:
# Show number of non_nan_values per column
df_no_nan.count()

fluidcog_age                175
crystalcog_age              174
earlychildcog_age           175
totalcog_age                175
dccs_age                    176
flanker_age                 176
lswmt_age                   176
orrt_age                    176
pcps_age                    176
picseq_age                  176
pvt_age                     176
EF_age                      176
age_years                   203
fluidcog_unadjusted         182
crystalcog_unadjusted       181
earlychildcog_unadjusted    182
totalcog_unadjusted         182
dccs_unadjusted             183
flanker_unadjusted          183
lswmt_unadjusted            183
orrt_unadjusted             183
pcps_unadjusted             183
picseq_unadjusted           183
pvt_unadjusted              183
EF_unadjusted               183
subject                     203
dtype: int64

In [15]:
# Drop all nan values
df_no_nan = df_no_nan.dropna()

In [16]:
# Add tda data to df_no_nan
same = tda['Subject'].isin(df_no_nan['subject'])
tda.drop(tda[~same].index, inplace = True)

In [17]:
# Sort tda data and reset index
tda.sort_values(by=['Subject'], inplace=True)
tda.reset_index(drop=True, inplace=True)
# Sort df and reset index
df_no_nan.sort_values(by=['subject'], inplace=True)
df_no_nan.reset_index(drop=True, inplace=True)

# Merge dataframes
print(df_no_nan.shape)
print(tda.shape)
df_no_nan = pd.concat([df_no_nan, tda], axis=1)
print(df_no_nan.shape)

(173, 26)
(173, 459)
(173, 485)


In [18]:
df_no_nan.to_csv('/mnt/resource/m.schepers/Data/TDA_Cog_Data/Females_train.csv', index=False)

In [19]:
test = pd.read_csv('/mnt/resource/m.schepers/Data/TDA_Cog_Data/Females_train.csv')
test.shape

(173, 485)